<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/european_call_cva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

class BinomialTreeBermudan:
    def __init__(self, S0, K, T, r, sigma, n_steps, exercise_dates):
        """
        S0: Initial stock price
        K: Strike price
        T: Time to maturity (in years)
        r: Risk-free interest rate
        sigma: Volatility
        n_steps: Number of time steps in the binomial tree
        exercise_dates: List of exercise dates (as fractions of T)
        """
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n_steps = n_steps
        self.exercise_dates = exercise_dates
        self.dt = torch.tensor(T/n_steps, dtype=torch.float32)
        self.u = torch.exp(sigma * torch.sqrt(self.dt))
        self.d = 1 / self.u
        self.p = (torch.exp(r * self.dt) - self.d) / (self.u - self.d)

    def build_tree(self):
        # Initialize asset prices at maturity
        self.S = torch.zeros(self.n_steps + 1, self.n_steps + 1)
        self.S[0, 0] = self.S0

        for i in range(1, self.n_steps + 1):
            self.S[0, i] = self.S[0, i - 1] * self.u
            for j in range(1, i + 1):
                self.S[j, i] = self.S[j - 1, i - 1] * self.d

    def option_price(self, option_type='call'):
        # Initialize option values at maturity
        self.V = torch.zeros_like(self.S)
        if option_type == 'call':
            self.V[:, -1] = torch.maximum(self.S[:, -1] - self.K, torch.tensor(0.0))
        elif option_type == 'put':
            self.V[:, -1] = torch.maximum(self.K - self.S[:, -1], torch.tensor(0.0))
        else:
            raise ValueError("option_type must be 'call' or 'put'")

        # Backward induction
        for i in range(self.n_steps - 1, -1, -1):
            for j in range(i + 1):
                self.V[j, i] = torch.exp(-self.r * self.dt) * (self.p * self.V[j, i + 1] + (1 - self.p) * self.V[j + 1, i + 1])

                # Check if current time step is an exercise date
                if i * self.dt in self.exercise_dates:
                    if option_type == 'call':
                        exercise_value = torch.maximum(self.S[j, i] - self.K, torch.tensor(0.0))
                    else:
                        exercise_value = torch.maximum(self.K - self.S[j, i], torch.tensor(0.0))
                    self.V[j, i] = torch.maximum(self.V[j, i], exercise_value)

        return self.V[0, 0]

    def calculate_greeks(self, option_type='call'):
        # Calculate Delta
        delta = (self.V[0, 1] - self.V[1, 1]) / (self.S[0, 1] - self.S[1, 1])

        # Calculate Gamma
        gamma = ((self.V[0, 2] - self.V[1, 2]) / (self.S[0, 2] - self.S[1, 2]) -
                 (self.V[1, 2] - self.V[2, 2]) / (self.S[1, 2] - self.S[2, 2])) / (0.5 * (self.S[0, 2] - self.S[2, 2]))

        # Calculate Theta
        theta = (self.V[0, 1] - self.V[0, 0]) / self.dt

        # Calculate Vega (using a small change in volatility)
        sigma_shift = 0.01
        tree_shifted = BinomialTreeBermudan(self.S0, self.K, self.T, self.r, self.sigma + sigma_shift, self.n_steps, self.exercise_dates)
        tree_shifted.build_tree()
        price_shifted = tree_shifted.option_price(option_type)
        vega = (price_shifted - self.option_price(option_type)) / sigma_shift

        # Calculate Rho (using a small change in interest rate)
        r_shift = 0.01
        tree_shifted = BinomialTreeBermudan(self.S0, self.K, self.T, self.r + r_shift, self.sigma, self.n_steps, self.exercise_dates)
        tree_shifted.build_tree()
        price_shifted = tree_shifted.option_price(option_type)
        rho = (price_shifted - self.option_price(option_type)) / r_shift

        return delta, gamma, theta, vega, rho

# Example usage
S0 = 100.0
K = 100.0
T = 1.0
r = 0.05
sigma = 0.2
n_steps = 100
exercise_dates = [0.25, 0.5, 0.75]

tree = BinomialTreeBermudan(S0, K, T, r, sigma, n_steps, exercise_dates)
tree.build_tree()
option_price = tree.option_price(option_type='call')
delta, gamma, theta, vega, rho = tree.calculate_greeks(option_type='call')

print(f"Option Price: {option_price:.4f}")
print(f"Delta: {delta:.4f}")
print(f"Gamma: {gamma:.4f}")
print(f"Theta: {theta:.4f}")
print(f"Vega: {vega:.4f}")
print(f"Rho: {rho:.4f}")

In [ ]:
import torch
import numpy as np

class CIRModel:
    def __init__(self, kappa, theta, sigma, r0, T, n_steps):
        """
        kappa: Speed of mean reversion
        theta: Long-term mean rate
        sigma: Volatility
        r0: Initial interest rate
        T: Time horizon
        n_steps: Number of time steps
        """
        self.kappa = kappa
        self.theta = theta
        self.sigma = sigma
        self.r0 = r0
        self.T = T
        self.n_steps = n_steps
        #self.dt = T / n_steps
        self.dt = torch.tensor(T/n_steps, dtype=torch.float32)

    def simulate(self, n_paths):
        # Simulate interest rate paths
        rates = torch.zeros(n_paths, self.n_steps + 1)
        rates[:, 0] = self.r0

        for t in range(1, self.n_steps + 1):
            dW = torch.randn(n_paths) * torch.sqrt(self.dt)
            rates[:, t] = rates[:, t - 1] + self.kappa * (self.theta - rates[:, t - 1]) * self.dt + \
                          self.sigma * torch.sqrt(rates[:, t - 1]) * dW
            rates[:, t] = torch.max(rates[:, t], torch.tensor(0.0))  # Ensure rates are non-negative

        return rates

class BucketingIntensityModel:
    def __init__(self, lambda_t, buckets):
        """
        lambda_t: Default intensity function (hazard rate)
        buckets: Time buckets for default probabilities
        """
        self.lambda_t = lambda_t
        self.buckets = buckets

    def survival_probability(self, t):
        # Calculate survival probability up to time t
        return torch.exp(-self.lambda_t * t)

    def default_probability(self, t1, t2):
        # Calculate default probability between t1 and t2
        return self.survival_probability(t1) - self.survival_probability(t2)

class CVA:
    def __init__(self, recovery_rate, discount_curve, exposure_profile, default_probabilities):
        """
        recovery_rate: Recovery rate in case of default
        discount_curve: Discount factors for each time bucket
        exposure_profile: Expected exposure at each time bucket
        default_probabilities: Default probabilities for each time bucket
        """
        self.recovery_rate = recovery_rate
        self.discount_curve = discount_curve
        self.exposure_profile = exposure_profile
        self.default_probabilities = default_probabilities

    def calculate(self):
        # Calculate CVA
        cva = (1 - self.recovery_rate) * torch.sum(
            self.exposure_profile * self.default_probabilities * self.discount_curve
        )
        return cva

# Example usage
if __name__ == "__main__":
    # Parameters
    S0 = 1.0
    K = 0.9
    T = 3.0
    r0 = 0.15
    kappa = 0.1
    theta = 0.05
    sigma = 0.2
    n_steps = 100
    n_paths = 1000
    recovery_rate = 0.4
    lambda_t = 0.02  # Constant hazard rate
    buckets = torch.linspace(0, T, n_steps + 1)

    # Simulate interest rates using CIR model
    cir_model = CIRModel(kappa, theta, sigma, r0, T, n_steps)
    rates = cir_model.simulate(n_paths)

    # Calculate discount factors
    discount_factors = torch.exp(-torch.cumsum(rates.mean(dim=0) * cir_model.dt, dim=0))

    # Calculate default probabilities using bucketing intensity model
    intensity_model = BucketingIntensityModel(lambda_t, buckets)
    default_probs = torch.tensor([
        intensity_model.default_probability(buckets[i], buckets[i + 1])
        for i in range(len(buckets) - 1)
    ])

    # Calculate expected exposure profile (simplified for demonstration)
    exposure_profile = torch.ones(len(buckets) - 1) * 10.0  # Placeholder for actual exposure

    # Calculate CVA
    cva_calculator = CVA(recovery_rate, discount_factors[:-1], exposure_profile, default_probs)
    cva = cva_calculator.calculate()

    print(f"CVA: {cva:.4f}")

In [ ]:
import torch

# Parameters for the European call option
S0 = 100.0  # Initial stock price
K = 90.0  # Strike price
T = 2.0  # Time to maturity (2 years)
r = 0.01  # Risk-free rate
sigma = 0.25  # Volatility

# CVA and intensity model parameters
LGD = 0.6  # Loss given default (e.g., 60%)
lambda_0 = 1.0  # Initial hazard rate
k = 0.5  # Mean-reversion speed
mu = 1.0  # Long-term mean of intensity
nu = 0.25  # Volatility of intensity
n_paths = 10000  # Number of Monte Carlo simulations
time_steps = 100  # Number of time steps in the simulation



# Generate time grid
dt = T / time_steps
time_grid = torch.linspace(0, T, time_steps)

# Simulate stock price paths using Geometric Brownian Motion (GBM)
# Simulate stock price paths using Geometric Brownian Motion (GBM)
def simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths):
    dt = T / time_steps
    # Convert dt to a PyTorch tensor
    dt = torch.tensor(dt, dtype=torch.float32)
    Z = torch.randn(n_paths, time_steps)  # Random shocks
    increments = (r - 0.5 * sigma**2) * dt + sigma * torch.sqrt(dt) * Z
    increments = torch.cat([torch.zeros(n_paths, 1), increments], dim=1)  # Initial price
    log_S = torch.cumsum(increments, dim=1)
    S = S0 * torch.exp(log_S)
    return S

# Simulate default times using an intensity model
def simulate_default_times(lambda_0, time_steps, T, n_paths):
    dt = T / time_steps
    # Convert lambda_0 and dt to tensors
    lambda_0 = torch.tensor(lambda_0, dtype=torch.float32)
    dt = torch.tensor(dt, dtype=torch.float32)
    default_probs = 1 - torch.exp(-lambda_0 * dt)  # Default probabilities per step
    defaults = torch.rand(n_paths, time_steps) < default_probs  # Simulate defaults
    # Convert defaults to integer type before applying argmax
    default_times = torch.argmax(defaults.type(torch.int), dim=1) * dt  # First default time
    default_times[default_times == 0] = T + 1  # No default
    return default_times


# Calculate CVA
def calculate_cva(LGD, S, K, T, r, lambda_0, default_times, time_grid):
    # Compute option payoff at maturity
    payoffs = torch.maximum(S[:, -1] - K, torch.tensor(0.0))  # Max(S_T - K, 0)

    # Discount factor to maturity
    discount_factors = torch.exp(-torch.tensor(r, dtype=torch.float32) * torch.tensor(T, dtype=torch.float32)) # Convert r and T to tensors

    # Compute survival probabilities
    survival_probs = torch.exp(-lambda_0 * time_grid[-1])

    # Compute CVA
    cva = LGD * torch.mean((1 - survival_probs) * discount_factors * payoffs)
    return cva.item()

# Simulate stock prices and default times
S = simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths)
default_times = simulate_default_times(lambda_0, time_steps, T, n_paths)

# Calculate CVA
cva = calculate_cva(LGD, S, K, T, r, lambda_0, default_times, time_grid)

print(f"Calculated CVA: {cva:.4f}")